### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session

* login (ssh) into the VM:
```
loginvm0011@ghiordanXRgpuvm:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```



  
   
* then go to:    
http://ghiordanxrgpuvm.eastus2.cloudapp.azure.com:9000/ 
  
  
Note: continuumio/miniconda3:4.5.4 is the latest tag as of 9/7/2018

In [45]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [46]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [47]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/aml-sdk_docker"
image_version = 'sdk.v.0.1.80'

Overwriting .env


In [48]:
%dotenv
docker_file_location = 'docker'
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_version')

In [49]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

!mkdir -p {docker_file_dir}
! chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_sdk.v.0.1.80'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml'

total 0


In [50]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 
  - numpy
  - cython
  - notebook=5.6.0 
  - nb_conda
  - scikit-learn
  - pip:
    - python-dotenv
#  - azureml-sdk[notebooks,automl]

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/aml_sdk_conda_dep_file.yml


In [51]:
# %%writefile $docker_file_path 

# FROM debian:latest
# MAINTAINER George Iordanescu <ghiordan@microsoft.com>
# # https://hub.docker.com/r/continuumio/miniconda3/~/dockerfile/

# ENV LANG=C.UTF-8 LC_ALL=C.UTF-8
# ENV PATH /opt/conda/bin:$PATH

# RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
#     wget bzip2 ca-certificates curl git && \
#     apt-get clean && \
#     rm -rf /var/lib/apt/lists/*
    
# # # Install Azure CLI
# # RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ xenial main" | \
# #     tee /etc/apt/sources.list.d/azure-cli.list && \
# #     curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
# #     apt-get update && \
# #     apt-get install -y --no-install-recommends \
# #     azure-cli


# ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml

# ENV CONDA_ENV_NAME aml-sdk-conda-env

# # RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
# RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ~/miniconda.sh && \
#     chmod +x ~/miniconda.sh && \
#     ~/miniconda.sh -b -p /opt/conda && \
#     rm ~/miniconda.sh 
    
# RUN  conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
#     conda clean --yes --all
# ENV PATH /opt/conda/envs/$CONDA_ENV_NAME/bin:$PATH

# RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && pip install azureml-sdk[notebooks]"]    
# # install and enable experiment widget
# # https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment
# RUN ["/bin/bash","-c","source activate aml-sdk-conda-env && \
#                     jupyter nbextension install --py --user azureml.train.widgets  && \
#                     jupyter nbextension enable --py --user azureml.train.widgets"]

# WORKDIR /workspace

# CMD [ "/bin/bash" ]

In [52]:
%%writefile $docker_file_path 

FROM continuumio/miniconda3:4.5.11
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    wget curl  \
    git && \ 
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*
    
ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml

ENV CONDA_ENV_NAME aml-sdk-conda-env

RUN  conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    conda clean --yes --all
ENV PATH /opt/conda/envs/$CONDA_ENV_NAME/bin:$PATH

#unclear if I can run pip from $CONDA_ENV_NAME env, but     
RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && pip install azureml-sdk[notebooks,automl] azureml-dataprep"] 

# install and enable experiment widget
# https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment
RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && \
                        jupyter nbextension install --py --user azureml.widgets  && \
                        jupyter nbextension enable --py --user azureml.widgets"]

# RUN [ "/bin/bash","-c","source activate aml-sdk-conda-env && \
#                         mkdir -p /workspace"]

CMD [ "/bin/bash","-c","source activate aml-sdk-conda-env"]

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_sdk.v.0.1.80


In [53]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + image_version_string


docker_file_path
docker_image_name
docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_sdk.v.0.1.80'

'georgedockeraccount/aml-sdk_docker:sdk.v.0.1.80'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker'

total 8
-rw-rw-r-- 1 loginvm0011 loginvm0011  218 Nov 20 18:57 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm0011 loginvm0011 1256 Nov 20 18:57 dockerfile_sdk.v.0.1.80


In [54]:
!echo docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache
!     docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache

docker build -t georgedockeraccount/aml-sdk_docker:sdk.v.0.1.80 -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker/dockerfile_sdk.v.0.1.80 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker --no-cache
Sending build context to Docker daemon  4.096kB
Step 1/10 : FROM continuumio/miniconda3:4.5.11
 ---> d3c252f8727b
Step 2/10 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 6062de3ca167
 ---> 368992ddc1f8
Removing intermediate container 6062de3ca167
Step 3/10 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     wget curl      git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in abb04caa0da4
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:3 http://deb.debian.org/debian stretch Release [118 kB]
Get:4 http://deb.debian.org/debian stretch Release.gpg [2434 B]
Get:5 http://security.debian.org/debian-security stretch/updates InRelease [94

 ---> e284891b8461
Removing intermediate container 5bd462b24686
Step 7/10 : ENV PATH /opt/conda/envs/$CONDA_ENV_NAME/bin:$PATH
 ---> Running in 06521af6effb
 ---> 785358da0a34
Removing intermediate container 06521af6effb
Step 8/10 : RUN /bin/bash -c source activate aml-sdk-conda-env && pip install azureml-sdk[notebooks,automl] azureml-dataprep
 ---> Running in bf25e99cf34e


  Running setup.py bdist_wheel for SecretStorage: started
  Running setup.py bdist_wheel for SecretStorage: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4e/5b/1b/be8c8a830a0243af85b2946a0aece2c6743d7f7f946977ed67
  Running setup.py bdist_wheel for pathspec: started
  Running setup.py bdist_wheel for pathspec: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Running setup.py bdist_wheel for tabulate: started
  Running setup.py bdist_wheel for tabulate: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
  Running setup.py bdist_wheel for antlr4-python3-runtime: started
  Running setup.py bdist_wheel for antlr4-python3-runtime: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ef/f6/18/ad300e691236a3408a99edc750484b56e8d6b11b2c38eacb10
  Running setup.py bdist_wheel

 ---> 52f952ae056e
Removing intermediate container bf25e99cf34e
Step 9/10 : RUN /bin/bash -c source activate aml-sdk-conda-env &&                         jupyter nbextension install --py --user azureml.widgets  &&                         jupyter nbextension enable --py --user azureml.widgets
 ---> Running in 11c09ae1de63
Installing /opt/conda/envs/aml-sdk-conda-env/lib/python3.6/site-packages/azureml/widgets/static -> azureml_widgets
Making directory: /root/.local/share/jupyter/nbextensions/azureml_widgets/
Copying: /opt/conda/envs/aml-sdk-conda-env/lib/python3.6/site-packages/azureml/widgets/static/index.js -> /root/.local/share/jupyter/nbextensions/azureml_widgets/index.js
Copying: /opt/conda/envs/aml-sdk-conda-env/lib/python3.6/site-packages/azureml/widgets/static/extension.js -> /root/.local/share/jupyter/nbextensions/azureml_widgets/extension.js
Making directory: /root/.local/share/jupyter/nbextensions/azureml_widgets/packages
Making directory: /root/.local/share/jupyter/nbextensi

In [55]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 0


In [56]:
!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/

#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [57]:
# !docker login -u=$docker_login -p=

In [58]:
!docker push $docker_image_name

The push refers to a repository [docker.io/georgedockeraccount/aml-sdk_docker]

b3af06d2: Preparing 
b707e3c2: Preparing 
8bc93611: Preparing 
57958a56: Preparing 
1c733412: Preparing 
b5c98f73: Preparing 
31f7d329: Preparing 
cc4bbc9d: Preparing 
ae060f2d: Preparing 


8bc93611: Pushed   1.381GB/1.335GBPushing  225.9MB/1.335GBPushing  356.2MB/1.335GBPushing  642.8MB/1.335GBsdk.v.0.1.80: digest: sha256:22b039124ca5424f652d01b536c3d4522ab96f375374b2539e850e58f8744d81 size: 2425


### To run aml SDK, ssh into a vm, run the command below and then go to: 
ghiordanxrgpuvm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [59]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $docker_image_name /bin/bash -c '"source activate aml-sdk-conda-env && jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root "'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker:sdk.v.0.1.80 /bin/bash -c "source activate aml-sdk-conda-env && jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root "


### Run a short test
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in a notebook

In [60]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [61]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [62]:
!ls -l $crt_local_dir

total 4
-rw-rw-r-- 1 loginvm0011 loginvm0011 982 Nov 20 19:03 test_aml_sdk_docker_image.ipynb


### Quick testing SDK docker image (see cell response "AML SDK version: <latest version>").
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [63]:
!docker run -v $crt_local_dir:/workspace:rw $docker_image_name /bin/bash -c "source activate aml-sdk-conda-env && jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 979 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [64]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.80\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


In [65]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 336781 bytes to createAMLSDKDocker.html
